In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch,torchvision
from torch.nn import *
from torch.optim import *
import wandb
import cv2
from tqdm import tqdm
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    LabelEncoder,
    Normalizer,
)
device = 'cuda'
PROJECT_NAME = 'Microsoft-Stock-Price-Prediction-Learning-RNN'
data = pd.read_csv('./data/data.csv')

In [2]:
len(data)

1511

In [3]:
data.head()

,Date,Open,High,Low,Close,Volume
0,4/1/2015 16:00:00,40.60,40.76,40.31,40.72,36865322
1,4/2/2015 16:00:00,40.66,40.74,40.12,40.29,37487476
2,4/6/2015 16:00:00,40.34,41.78,40.18,41.55,39223692
3,4/7/2015 16:00:00,41.61,41.91,41.31,41.53,28809375
4,4/8/2015 16:00:00,41.48,41.69,41.04,41.42,24753438


In [4]:
data = data['Open']

In [5]:
data = np.array(data.tolist()).reshape(1,-1)

In [6]:
data_input = torch.from_numpy(data[:3,:-1]).float().to(device)
data_target = torch.from_numpy(data[:3,1:]).float().to(device)

In [7]:
data_input

tensor([[ 40.6000,  40.6600,  40.3400,  ..., 231.5500, 236.5900, 233.5300]],
       device='cuda:0')

In [8]:
data_target

tensor([[ 40.6600,  40.3400,  41.6100,  ..., 236.5900, 233.5300, 232.9100]],
       device='cuda:0')

In [9]:
class Model(Module):
    def __init__(self,hidden=128):
        super().__init__()
        self.hidden = hidden
        self.lstm1 = LSTMCell(1,self.hidden).to(device)
        self.lstm2 = LSTMCell(self.hidden,self.hidden).to(device)
        self.lstm3 = LSTMCell(self.hidden,self.hidden).to(device)
        self.linear1 = Linear(self.hidden,1).to(device)
    
    def forward(self,X,future=0):
        preds = []
        batch_size = X.size(0)
        h_t1 = torch.zeros(batch_size,self.hidden,dtype=torch.float32)
        c_t1 = torch.zeros(batch_size,self.hidden,dtype=torch.float32)
        h_t2 = torch.zeros(batch_size,self.hidden,dtype=torch.float32)
        c_t2 = torch.zeros(batch_size,self.hidden,dtype=torch.float32)
        h_t3 = torch.zeros(batch_size,self.hidden,dtype=torch.float32)
        c_t3 = torch.zeros(batch_size,self.hidden,dtype=torch.float32)
        h_t1 = h_t1.to(device)
        c_t1 = h_t1.to(device)
        h_t2 = h_t2.to(device)
        c_t2 = h_t2.to(device)
        h_t3 = h_t3.to(device)
        c_t3 = h_t3.to(device)
        for X_batch in X.split(1,dim=1):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t1 = h_t1.to(device)
            c_t1 = h_t1.to(device)
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            h_t2 = h_t2.to(device)
            c_t2 = h_t2.to(device)
            h_t3,c_t3 = self.lstm3(h_t2,(h_t3,c_t3))
            h_t3 = h_t3.to(device)
            c_t3 = h_t3.to(device)
            pred = self.linear1(h_t3)
            preds.append(pred)
        for i in range(future):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t1 = h_t1.to(device)
            c_t1 = h_t1.to(device)
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            h_t2 = h_t2.to(device)
            c_t2 = h_t2.to(device)
            h_t3,c_t3 = self.lstm3(h_t2,(h_t3,c_t3))
            h_t3 = h_t3.to(device)
            c_t3 = h_t3.to(device)
            pred = self.linear1(h_t3)
            preds.append(pred)
        preds = torch.cat(preds,dim=1)
        return preds

In [10]:
model = Model().to(device)
optimizer = LBFGS(model.parameters(),lr=0.8)
criterion = MSELoss()

In [11]:
epochs = 100

In [12]:
# wandb.init(project=PROJECT_NAME,name='test')
# for _ in tqdm(range(epochs)):
#     def closure():
#         optimizer.zero_grad()
#         preds = model(data_input)
#         loss = criterion(preds,data_target)
#         wandb.log({'Loss':loss.item()})
#         loss.backward()
#         return loss
#     optimizer.step(closure)
#     with torch.no_grad():
#         future = 100
#         preds = model(data_input,future)
#         loss = criterion(preds[:,:-future],data_target)
#         wandb.log({'Val Loss':loss.item()})
#         preds = preds.cpu().detach().numpy()
#     plt.figure(figsize=(12,7))
#     batch_size = data_input.shape[1]
#     def draw(y):
#         plt.plot(np.arange(batch_size),data_target.cpu().view(-1),'b')
#         plt.plot(np.arange(batch_size,batch_size+future),y[batch_size:],'r')
#     draw(preds[0])
#     plt.savefig('./preds/img.png')
#     plt.close()
#     wandb.log({'Img':wandb.Image(cv2.imread('./preds/img.png'))})
# wandb.finish()

In [13]:
pres = [
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    LabelEncoder,
    Normalizer
]

In [14]:
lrs = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

In [15]:
optimizers = [Adam,AdamW,Adamax,SGD,LBFGS]

In [16]:
criterions = [MSELoss(),L1Loss()]

In [17]:
hiddens = [16,32,64,128,256,512,1024]

In [18]:
# I dont have time to tune this project so If you want to you can do that really easyly 